In [124]:
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import ta

df = fdr.DataReader('005930','2004-11-16', '2024-08-16')

In [125]:
# 볼린저밴드
bb = ta.volatility.BollingerBands(close=df['Close'], window=20, fillna=False)
df['PB'] = bb.bollinger_pband()
df['BandWidth'] = bb.bollinger_wband()

In [126]:
# DMI, ADX
# dmi = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=14, fillna=False)
# df['PDI'],df['MDI'],df['ADX'] = dmi.adx_pos(),dmi.adx_neg(),dmi.adx()

pdiff, ndiff = df['High'].diff(1), df['Low'].diff(1)
pdm = pd.Series(np.where((pdiff > 0) & (pdiff > -ndiff), pdiff, 0), index=df.index)
ndm = pd.Series(np.where((-ndiff > 0) & (-ndiff > pdiff), -ndiff, 0), index=df.index)

tr = pd.DataFrame()
tr['1'] = df['High'] - df['Low']
tr['2'] = (df['High'] - df['Close'].shift(-1)).abs()
tr['3'] = (df['Low'] - df['Close'].shift(-1)).abs()
tr = tr.max(axis = 1)

def sum_index(df,nums):
    sum = pd.Series(np.nan, index=df.index)
    sum.iloc[13] = nums[0:14].sum()
    for i in range(14,len(df)):
        sum.iloc[i] = sum.iloc[i-1] - sum.iloc[i-1]/14 + nums.iloc[i]
    return sum

pdmn, ndmn, trn = sum_index(df,pdm), sum_index(df,ndm), sum_index(df,tr)

df['PDI'], df['MDI'] = pdmn/trn*100, ndmn/trn*100
dx = ((df['PDI']-df['MDI']).abs()) / (df['PDI']+df['MDI']) * 100

adx = pd.Series(np.nan, index=df.index)
adx.iloc[26] = dx[13:27].mean()
for i in range(27,len(df)):
    adx.iloc[i] = (adx.iloc[i-1]*13 + dx.iloc[i])/14

df['ADX'] = adx

In [127]:
# WMA (가중이동평균)
df['CWMA'] = ta.trend.WMAIndicator(close=df['Close'], window=5, fillna=False).wma()
df['VWMA'] = ta.trend.WMAIndicator(close=df['Volume'], window=5, fillna=False).wma()

In [128]:
# MACD
df['MACD'] = ta.trend.macd(close=df['Close'], window_slow=26, window_fast=12, fillna=False)
df['MACD_sig'] = ta.trend.macd_signal(close=df['Close'], window_slow=26, window_fast=12, window_sign=9, fillna=False)

In [129]:
# RSI
df['RSI'] = ta.momentum.rsi(close=df['Close'], window=14, fillna=False)
df['RSI_sig'] = df['RSI'].rolling(9).mean()

In [130]:
# STOCHASTIC FAST
df['%K'] = ta.momentum.stoch(high=df['High'], low=df['Low'], close=df['Close'], window=14, smooth_window=3, fillna=False)
df['%D'] = ta.momentum.stoch_signal(high=df['High'], low=df['Low'], close=df['Close'], window=14, smooth_window=3, fillna=False)

# STOCAHSTIC SLOW (FAST와 SLOW 둘 중 하나를 반영해야 함)
# df['Slow_%K'] = df['%D']
# df['Slow_%D'] = df['Slow_%K'].rolling(3).mean()

In [134]:
df = df[33:]; df.tail()

,Open,High,Low,Close,Volume,Change,PB,BandWidth,PDI,MDI,ADX,CWMA,VWMA,MACD,MACD_sig,RSI,RSI_sig,%K,%D
Date,,,,,,,,,,,,,,,,,,,
2024-08-09,75700,75800,74200,74700,16388222,0.017711,0.185381,24.377061,19.726125,32.691508,24.093548,73840.000000,2.952870e+07,-2338.746968,-1241.965020,38.460203,39.574546,28.301887,25.576520
2024-08-12,75200,75900,74800,75500,9839259,0.010710,0.251606,24.276895,19.352107,31.675436,24.097605,74560.000000,2.084730e+07,-2319.645728,-1457.501162,40.593091,39.303713,33.333333,27.253669
2024-08-13,76500,76600,75500,76100,10716261,0.007947,0.303029,23.327003,20.267308,30.347132,23.798841,75206.666667,1.544286e+07,-2230.382412,-1612.077412,42.210815,38.084487,37.106918,32.914046
2024-08-14,77400,77800,77000,77200,13246168,0.014455,0.381179,22.183752,21.617454,27.969496,23.013918,75980.000000,1.332032e+07,-2047.280015,-1699.117932,45.159286,37.490015,44.025157,38.155136
2024-08-16,79400,80200,78700,80200,20804476,0.038860,0.576927,20.497977,26.882344,26.905371,21.373124,77586.666667,1.501484e+07,-1641.176655,-1687.529677,52.306297,38.764603,62.893082,48.008386


In [132]:
df.shape

(4851, 19)

In [133]:
df.to_csv('stock.csv',encoding='utf-8-sig')